<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:60px; font-weight:bold;">CineClassify</div>
</div>
<br>
<p>In een wereld waar films in overvloed zijn, is er een groeiende behoefte om ze gemakkelijk en duidelijk te kunnen classificeren. Veel kijkers hebben geen goed overzicht over alle mengende genres en de vele nuances. Dit kan het zoeken van een leuke film voor op de vrijdagavond vanuit de Netflix catalogus of voor een trip naar de bios onnodig gecompliceerd maken.  

Daarom heeft het nieuwe filmplatform “CineClassify” als doel om films automatisch te laten classificeren in verschillende genres en een duidelijke gids te brengen naar de kijkers. Om dit doel te bereiken hebben ze een data-science team ingehuurd om een model te maken die genres kan gaan voorspellen. Aan de hand van verschillende gegevens, bijv. cast, regisseur, reviews, etc., moet het mogelijk worden gemaakt om filmliefhebbers over de wereld te helpen om eenvoudiger films te vinden die passen bij hun smaak. 

In dit notebook werken we aan onze opdracht van CineClassify. In dit notebook zal er een pipeline worden gebouwd om de data in te laden en er een dataframe van te maken. Deze dataframe kan worden gebruikt om de data duidelijk in te zien voor het datascience team.</p>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Inhoudsopgave</div>
    <a name='begin'></a>
</div>

1. [Importeren van libaries](#start)
2. [IMDb Webscraping](#ws)
3. [Database: Movie Summaries](#db)
4. [API](#api)
5. [Preprocessing en Feature Engineering](#tr)
6. [Opzetten van de Pipeline](#pipe)
7. [Aantonen dat de Pipeline werkt](#toon)

<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Importeren van libaries</div>
    <a name='start'></a>
</div>

In [3]:
# Importeren standaard libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importeren webscraping libaries
import requests
from bs4 import BeautifulSoup
import regex as re

# Importeren time-out libaries
from time import sleep
from random import randint

# Importeren Database libaries
import tarfile
import os
import urllib.request
from io import BytesIO
import sqlite3

# Importeren FE libaries
import re
from nltk.corpus import stopwords

# Importeren API libabries
import requests

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">IMDb Webscraping</div>
    <a name='ws'></a>
</div>

Om een deel van de data te krijgen is het nodig om data te verkrijgen van het internet. Dit wordt gedaan door middel van een techniek genaamd webscraping. Door middel van de BeautifulSoup library voor Python is het gemakkelijk gemaakt om deze stappen te ondernemen. Door het bekijken van de HTML code van de website kunnen de nodige elementen gevonden worden en kan de data opgehaald worden van de website. Voordat we beginnen met het coderen van de soup worden er eerst een paar nodige elementen aangemaakt.

Om deze data te verkrijgen is er gebruik gemaakt van een webscraper. Deze webscraper is helaas niet meer bruikbaar doordat de source-code van de website volledig is veranderd. De code die gebruikt is om de data te verkrijgen is in de markdown cell gezet. Na de uitleg van de code lezen we het csv bestand in dat gemaakt is na het uitvoeren van de code.

```py
# Aanmaken van lijsten om de data in te stoppen
titel = []
jaartal = []
lengte = []
imdb_scores = []
meta_scores = []
stemmen = []
us_omzet = []
omschrijving = []
certificaat = []
genre = []
regisseur = []
sterren = []

# Verkrijgen van de engelse namen van films
en_titel = {'Accept-Language': 'en-US, en;q=0.5'}

# Aanmaken van lijst voor pagina's
pagina = np.arange(1, 1001, 50)

# Aanmaken van de URL
url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start='

# Zorgen dat de scraping voor elke 50 gaat
for p in pagina:
    # Pakken van URL
    p = requests.get(
        url + str(p) + '&ref_=adv_nxt', headers=en_titel
        )

    # Beginnen van de soup
    soup = BeautifulSoup(p.text, 'html.parser')

    # Zoeken van alle films op de pagina
    films = soup.find_all('div', class_='lister-item mode-advanced')

    # Wachtijd van 2 tot 10 seconden
    sleep(randint(2, 10))

    for item in films:
        # Titel
        titel.append(item.h3.a.text)

        # Jaartal
        jaartal.append(item.h3.find('span', class_='lister-item-year').text)

        # Regisseur
        regisseur.append(item.find('p', class_='').find('a').text)

        # Hoofd-acteurs
        acteurs = item.find('p', class_='').find_all('a')
        stars = []
        for tag in acteurs[-4:]:
            stars.append(tag.text)
        sterren.append(stars)

        # Leeftijd certificatie
        cert = (item.find('span', class_='certificate').text
                if item.p.find('span', class_='certificate') else 'NotFound')
        certificaat.append(cert)

        # Lengte
        runtime = (item.find('span', class_='runtime').text
                    if item.p.find('span', class_='runtime') else 'NotFound')
        lengte.append(runtime)

        # Genre
        gen = (item.find('span', class_='genre').text
                if item.p.find('span', class_='genre') else 'NotFound')
        genre.append(gen)

        # IMDb rating
        imdb_scores.append(float(item.strong.text))

        # meta_scores
        m_score = (item.find('span', class_='metascore').text
                    if item.find('span', class_='metascore') else 'NotFound')
        meta_scores.append(m_score)

        # Omschrijving
        desc = item.find_all('p', class_='text-muted')
        omschrijving.append(desc[1].text)

        # Stemmen en Omzet
        so = item.find_all('span', attrs={'name':'nv'})
        stemmen.append(so[0].text)
        us_omzet.append(so[1].text if len(so) > 1 else '-')

    print("-- Iteratie van loop voltooid --")

# Aanmaken van een dataframe
films = pd.DataFrame(
    {'Film' : titel,
     'Omschrijving' : omschrijving,
     'Regisseur' : regisseur,
     'Hoofd Acteurs' : sterren,
     'Age_Rating' : certificaat,
     'Genres' : genre,
     'Jaar' : jaartal,
     'Minuten' : lengte,
     'IMDb_Score' : imdb_scores,
     'Meta_Score' : meta_scores,
     'Stemmen' : stemmen,
     'Omzet (in M)' : us_omzet}
)

# Data preprocessing van de films dataframe
# Opschonen van de omschrijving kolom
films['Omschrijving'] = films['Omschrijving'].str.strip()

# Opschonen van de acteurs kolom
films['Hoofd Acteurs'] = films['Hoofd Acteurs'].astype(str)\
                            .replace({'\'': '', '\[|\]': ''}, regex=True)

# Opschonen van de genres kolom
films['Genres'] = films['Genres'].str.strip()

# Opschonen van de Jaar kolom
films['Jaar'] = films['Jaar'].str.extract('(\d+)').astype(int)

# Opschonen van de Minuten kolom
films['Minuten'] = films['Minuten'].str.extract('(\d+)').astype(int)

# Opschonen van de Meta_score kolom
films['Meta_Score'] = films['Meta_Score'].str.extract('(\d+)')

# Omzetten naar float en NotFound veranderen naar NaN
films['Meta_Score'] = pd.to_numeric(films['Meta_Score'], errors='coerce')

# Opschonen van de Stemmen kolom
films['Stemmen'] = films['Stemmen'].str.replace(',', '').astype(int)

# Opschonen van Omzet kolom
# Weghalen van '$' en 'M'
films['Omzet (in M)'] = films['Omzet (in M)'].map(lambda x: x.lstrip('$').rstrip('M'))

# Omzetten naar float en NotFound veranderen naar NaN
films['Omzet (in M)'] = pd.to_numeric(films['Omzet (in M)'], errors='coerce')

films.to_csv('IMDb_data.csv')
```

Met de bovenstaande code is het gelukt om de data van de voorgaande versie van IMDb te extraheren en de data alvast te preprocessen. Deze data is vervolgens in een CSV bestand gezet die nu zal worden ingeladen.

Later is besloten om enkel drie kolommen te behouden in deze dataset. Daarom missen er heel wat kolommen uit de oorspronkelijke code met het bestand. De code om deze kolommen eruit te halen is te vinden in het hoofdstuk preprocessing.

In [2]:
# Inladen van IMDb Data
films = pd.read_csv('IMDb_data.csv')

# Tonen data
display(films.dtypes)
display(films)

Film            object
Omschrijving    object
Genres          object
dtype: object

,Film,Omschrijving,Genres
0,The Shawshank Redemption,"Over the course of several years, two convicts...",Drama
1,The Godfather,"Don Vito Corleone, head of a mafia family, dec...","Crime, Drama"
2,The Dark Knight,When the menace known as the Joker wreaks havo...,"Action, Crime, Drama"
3,Schindler's List,"In German-occupied Poland during World War II,...","Biography, Drama, History"
4,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,"Action, Adventure, Drama"
...,...,...,...
995,When Marnie Was There,"Due to 12-year-old Anna's asthma, she's sent t...","Animation, Drama, Family"
996,Control,"A profile of Ian Curtis, the enigmatic singer ...","Biography, Drama, Music"
997,Philomena,A world-weary political journalist picks up th...,"Biography, Comedy, Drama"
998,Shine,"Pianist David Helfgott, driven by his father a...","Biography, Drama, Music"


Met deze dataframe is het mogelijk om stappen voor het feature engineering te testen, voordat deze in de pipeline komen.

[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Database: Movie Summaries</div>
    <a name='db'></a>
</div>

Voor het Database onderdeel van de opgave, is er gebruik gemaakt van een online downloadbare bron genaamd de Movie Summary Corpus. Deze databron bevat verschillende informatie van ongeveer 42000 films. Er staat informatie over de verschillende karakters in de films, een uitgebreide omschrijving van het plot en metadata over de film. Voor dit onderzoek zal de data over de karakters niet nodig zijn en dus zal deze niet worden ingeladen. De onderstaande stukken uitleg zijn voor de data die wij gaan gebruiken. Met dank aan de README.txt die bij de download van de data zat, is het gemakkelijker om de data en de structuur te begrijpen. Voor het inladen van elk bestand is de README gebruikt om de kolomnamen aan te maken en de data op de juiste manier in te laden.
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: plot_summaries.txt</div>
    <a name='db'></a>
</div>

Door middel van pd.read_csv is het mogelijk om text bestanden in te lezen. Bij het bekijken van het tekst bestand werd duidelijk dat de scheiding van kolommen is aangegeven door een tab. Verder is de README informatie gebruikt om de kolomnamen te maken:
- Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary. (Gekopieerd van README.txt)
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: movie.metadata.tsv</div>
</div>

Ook bij dit tsv bestand kan er gebruik worden gemaakt van de pd.read_csv functie. Bij het lezen van de README is duidelijk neergezet hoe de structuur van dit bestand eruit ziet.

Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns: (Gekopieerd van README.txt)

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

<br>
<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Data inladen naar database</div>
</div>

Om de data in te kunnen lezen, wordt er gebruik gemaakt van een database. Deze database zal worden aangemaakt door de corpus van het internet te downloaden, om deze vervolgens in een database in te laden. Deze taken worden gedaan met behulp van verschillende libaries, met name urllib en sqlite3. Om al deze stappen uit te voeren worden er eerst een aantal functies aangemaakt. Elke functie heeft zijn eigen doel en samen geven ze een gestreamlinede weg naar het maken van de database.

In [3]:
def download_and_extract(url, folder):
    """
    Deze functie download een gecomprimeerd archief van de
    opgegeven URL en extraheerd de data naar een opgegeven map.

    Parameters:
    ----------
    archive_url : str
        De URL van het te downloaden archief.

    target_folder : str
        Het pad naar de doelmap waarin de
        bestanden worden geëxtraheerd.

    Returns:
    ----------
    None
    """
    # Openen van de URL en lezen van de response
    response = urllib.request.urlopen(url)
    tar_data = BytesIO(response.read())

    # Downloaden en extraheren van de bestanden
    with tarfile.open(fileobj=tar_data, mode='r:gz') as tar:
        tar.extractall(path=folder)

def transfer_dataframe_to_db(df, table_name, db_path):
    """
    Overzetten van een Pandas DataFrame naar een
    SQLite-database tabel.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het DataFrame dat moet worden overgezet.

    table_name : str
        De naam van de tabel in de database.

    db_path : str
        Het pad naar de SQLite-database.

    Returns:
    ----------
    None
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_path)

    # Aanmaken van de tabel
    df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

    # Pushen van verandering naar Database en stoppen connectie
    conn.commit()
    conn.close()

def query_exe(db_path, query):
    """
    Uitvoeren van een SQL-query op de opgegeven SQLite-database
    en de resultaten omzetten naar een Pandas DataFrame.

    Parameters:
    db_path : str
        Het pad naar de SQLite-database.

    query : str
        De SQL-query die moet worden uitgevoerd.

    Returns:
    ----------
    df : pandas.DataFrame
        Het resultaat van de query als een DataFrame.
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_path)

    # Query inlezen tot dataframe
    df = pd.read_sql_query(query, conn)

    # Sluiten van de connectie met database
    conn.close()

    return df

Met de functies aangemaakt kunnen we de nodige parameters definieeren om de data te downloaden en in te laden in de database. 

In [4]:
# URL download
url = "http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz"

# Folder path
folder = "MovieSummaries"

# Database naam
database = 'movie_database.db'

# Kolommen van de data
kolommen = {
    "movie.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Film',
                       'Release_date', 'Box_office', 'Lengte',
                       'Talen', 'Landen', 'Genres'],
    "character.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Release_date',
                           'Char_Name', 'Actor_DoB', 'Actor_gender', 'Actor_h',
                           'Actor_eth', 'Actor_Name', 'Actor_age',
                           'Freebase_map_ID', 'Freebase_ch_ID', 'Freebase_ac_ID'],
    "plot_summaries": ['Wikipedia_ID', 'Summary'],
    "name.clusters": ['Name', 'Freebase_ID'],
    "tvtropes.clusters": ['Trope', 'Character_Data']
}

Met gebruik van deze vier parameters is het mogelijk om de nodige functies voor het downloaden van de data en het inladen in de database uit te gaan voeren.

In [5]:
# Uitvoeren van de download en extract stap
download_and_extract(url, folder)

# Loopen over de folder met geextraheerde data
for root, dirs, files in os.walk(folder):
    for file in files:
        # Aanmaken van bestand locatie, type en de tabel naam
        file_path = os.path.join(root, file)
        file_type = 'tsv' if file.endswith(".tsv") else 'txt'
        table_name = os.path.splitext(file)[0]

        # Excluderen van README.txt
        if file != "README.txt":
            print(f"Processen van {file_type.upper()} bestand: {file_path}")

            # Aangeven welke kolommen worden gebruikt
            custom_kols = kolommen.get(table_name, None)

            # Aanmaken van dataframe
            df = pd.read_csv(file_path, sep='\t', names=custom_kols)

            # Dataframe transporteren naar database
            transfer_dataframe_to_db(df, table_name, database)
            print(f"Tabel '{table_name}' is toegevoegd")

Processen van TSV bestand: MovieSummaries\MovieSummaries\character.metadata.tsv
Tabel 'character.metadata' is toegevoegd
Processen van TSV bestand: MovieSummaries\MovieSummaries\movie.metadata.tsv
Tabel 'movie.metadata' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\name.clusters.txt
Tabel 'name.clusters' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\plot_summaries.txt
Tabel 'plot_summaries' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\tvtropes.clusters.txt
Tabel 'tvtropes.clusters' is toegevoegd


Nu alle data als tabel is toegevoegd in de database, worden er een paar queries uitgevoerd. Op deze manier is het aan te tonen dat de code heeft gewerkt.

In [6]:
# Aanmaken van query
query = """
    SELECT Wikipedia_ID,
           Film,
           Genres
        FROM 'movie.metadata'
"""

# Aanmaken van dataframe door query in te lezen
df = query_exe(database, query)

# Tonen van dataframe
display(df)

,Wikipedia_ID,Film,Genres
0,975900,Ghosts of Mars,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,"{""/m/07s9rl0"": ""Drama""}"
...,...,...,...
81736,35228177,Mermaids: The Body Found,"{""/m/07s9rl0"": ""Drama""}"
81737,34980460,Knuckle,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,Another Nice Mess,"{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,The Super Dimension Fortress Macross II: Lover...,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [7]:
# Aanmaken van query
query = """
    SELECT *
        FROM 'plot_summaries'
"""

# Aanmaken van dataframe door query in te lezen
df_sum = query_exe(database, query)

# Tonen van dataframe
display(df_sum)

,Wikipedia_ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


Met deze dataframes is het mogelijk om stappen voor het preprocessen en feature engineering te testen, voordat deze in de pipeline komen.

[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">API</div>
    <a name='api'></a>
</div>

Voor het API onderdeel van de opgave, is er gebruik gemaakt van een gratis move api van de website Rapid api. Deze Api bevat meer dan 9 miljoen verschillende titles van films, series en afleveringen. De titels worden weekelijks geupdate en de ratings en afleveringen worden dagelijks bijgewerkt. In de api staat er informatie over de films en series waaronder de cast, de awards die gwonnen zijn, het jaar dat het uitkwam, de rating (van de IMDB-website), het genre en een korte omschrijving van de film/serie. Wij gebruiken alleen de titel, het genre en een korte omschrijving van de film dus hier beneden zie je de code over hoe wij deze uit de api data hebben gehaald. 

In [4]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

querystring = {"limit":"50","info":"base_info"} #limit is 50 want de api kan per keer maar 50 titels ophalen

headers = {
	"X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


resultaten = []

# Itereer over elk resultaat in de 'results' lijst van de JSON-respons
for resultaat in response.json()['results']:
    # Haal de benodigde velden op
    titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan
    genres = [genre['text'] for genre in resultaat['genres']['genres']] if 'genres' in resultaat else np.nan
    plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

    # Voeg de resultaten toe aan de lijst
    resultaten.append({
        'Titel': titel,
        'Genre': genres,
        'Beschrijving': plot
    })

df = pd.DataFrame(resultaten)

#verwijder de missende waardes
df.dropna(subset=['Titel', 'Genre', 'Beschrijving'], inplace=True)

# Toon de DataFrame
display(df)


,Titel,Genre,Beschrijving
1,Les blanchisseuses,[Short],This lost film presumably features women washi...
2,Dessinateur: Von Bismark,[Short],This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest","[Short, Sport]",Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,[Short],"Little is known about this lost film, the thir..."
7,Cripple Creek Bar-Room Scene,"[Western, Short]",A vignette of a barroom/liquor-store in the We...
11,Séance de prestidigitation,[Short],Boognish the goddemon Wants your soul to perfo...
15,L'hallucination de l'alchimiste,"[Short, Fantasy, Horror]",Misidentified as Alchimiste Parafaragaramus ou...
16,Une partie de cartes,"[Short, Biography]",In what is considered to be the first remake i...
21,Escamotage d'une dame au théâtre Robert Houdin,"[Short, Horror]",As an elegant maestro of mirage and delusion d...
22,Campement de bohémiens,"[Documentary, Short]",Very little is known of this lost film; accord...


In [5]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}


resultaten = []

# Itereer over de paginanummers
for page_number in range(1, 5):
    
    querystring = {
        "page": str(page_number),
        "info": "base_info",
        "limit": "50"
    }

    response = requests.get(url, headers=headers, params=querystring)

    # Itereer over elk resultaat in de 'results' lijst van de JSON-respons
    for resultaat in response.json()['results']:
        # Haal de benodigde velden op
        titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan
        genres = [genre['text'] for genre in resultaat['genres']['genres']] if 'genres' in resultaat else np.nan
        plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

        # Voeg de resultaten toe aan de lijst
        resultaten.append({
            'Titel': titel,
            'Genre': genres,
            'Beschrijving': plot
        })

df2 = pd.DataFrame(resultaten)

#verwijder de missende waardes
df2.dropna(subset=['Titel', 'Genre', 'Beschrijving'], inplace=True)

# Toon de DataFrame
display(df2)

,Titel,Genre,Beschrijving
1,Les blanchisseuses,[Short],This lost film presumably features women washi...
2,Dessinateur: Von Bismark,[Short],This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest","[Short, Sport]",Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,[Short],"Little is known about this lost film, the thir..."
7,Cripple Creek Bar-Room Scene,"[Western, Short]",A vignette of a barroom/liquor-store in the We...
...,...,...,...
193,Distributing a War Extra,"[Short, Documentary]","""This scene shows a crowd of newsboys running ..."
194,Gran corrida de toros,"[Documentary, Short, Sport]",Short documentary about the bullfighter Luis M...
195,Photographing a Ghost,"[Short, Horror]",Photographer tries to take a picture of a ghos...
197,Cendrillon,"[Short, Drama, Family, Fantasy, Romance]",A fairy godmother magically turns Cinderella's...


[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Preprocessing en Feature Engineering</div>
    <a name='tr'></a>
</div>

Om te kunnen werken met de ingeladen data is het noodzakelijk dat er toepassingen op worden uitgevoerd. De eerste vorm van toepassingen heet preprocessing. Deze stap is verantwoordelijk voor het volledig opschonen van de data. De tweede vorm heet Feature engineering. Deze vorm van toepassingen slaan op het bruikbaar maken van de data voor Machine Learning. Hierbij moeten er vaak nieuwe kolommen worden aangemaakt, of moeten oude kolommen worden herschreven. In deze sectie van het notebook gaan we kijken naar welke transformaties de data nodig zal hebben om bruikbaar te zijn voor Machine Learning.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Preprocessing</div>
</div>

Omdat veel data vanuit ruwe bronnen niet correct of compleet is, is het nodig om de data waar nodig aan te passen. Deze eerste aanpassingen zijn de stappen voor preprocessing. In dit process is het gebruikelijk dat de data volledig wordt opgeschoond en dat er eventuele technieken worden gebruikt om bepaalde data typen beter bruikbaar te maken. Om hiermee te beginnen zal er worden gekeken naar missende waarden in onze datasets.

In [8]:
def data_info(df):
    """
    Deze functie maakt een dataframe waarbij er een soort omschrijving komt
    over de data in elk dataframe.

    Parameters:
    ----------
    df : pd.DataFrame
        Het dataframe waarbij de kolommen worden bekeken

    Returns:
    ----------
    info : pd.DataFrame
        Een dataframe met de volgende informatie:
            - Aantal missende waarden
            - Percentage missende waarden
            - Aantal nulwaarden voor numerieke kolommen
            - Type data (numeriek of categorie)
            - Hoeveelheid categorieen
    """
    # Maak een dataframe met missende waarden
    info = pd.DataFrame(df.isnull().sum(), columns=['Missende_waarden'])

    # Voeg een kolom toe met percentage missende waarden
    info['Perc_missend'] = round(info['Missende_waarden'] / len(df) * 100, 2)

    # Voeg een kolom toe voor data type
    types = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            types.append('Numeriek')
        else:
            types.append('Categorie')
    info['Type'] = types

    # Voeg een kolom toe met aantal nulwaarden
    nulwaarden = []

    # For-Loop om te kijken voor data type
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            nulwaarden.append((df[col] == 0).sum())
        else:
            nulwaarden.append('-')
    info['Nulwaarden'] = nulwaarden

    # Voeg een kolom toe voor aantal categorieen
    cat_aantal = []

    # For-Loop om te kijken voor data type
    for col in df.columns:
        if info.loc[col, 'Type'] == 'Categorie':
            cat_aantal.append(df[col].nunique())
        else:
            cat_aantal.append('-')
    info['Aantal_Categorie'] = cat_aantal

    return display(info)


Aan de hand van deze functie kan er overzichtelijk worden gekeken naar de missende waarden van verschillende dataframes. Dit overzicht kan worden gebruikt om te bepalen wat we met de missende willen gaan doen.

In [9]:
# Tonen van missende waarden per dataframe
data_info(df)
data_info(df_sum)
data_info(films)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Wikipedia_ID,0,0.0,Numeriek,0,-
Film,0,0.0,Categorie,-,75478
Genres,0,0.0,Categorie,-,23817


,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Wikipedia_ID,0,0.0,Numeriek,0,-
Summary,0,0.0,Categorie,-,42295


,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Film,0,0.0,Categorie,-,994
Omschrijving,0,0.0,Categorie,-,1000
Genres,0,0.0,Categorie,-,194


Het dataframe met missende waarden is het dataframe 'films'. Hierbij gaat het om de kolommen 'Omzet (in M)' en 'Meta_Score'. Echter zijn alleen de kolommen 'Film', 'Omschrijving' en 'Genres' van belang. Dit houd in dat de rest gedropped kan worden uit het CSV bestand. Dit zal buiten de pipeline om gebeuren, aangezien dit uit de code was gehaald indien deze nog functioneerde.

In [10]:
# Lijst voor kolommen om te behouden
cols_to_keep = ['Film', 'Omschrijving', 'Genres']

# Alleen behouden van de nodige kolommen
films = films[cols_to_keep]

# Tonen van de aanpassingen
films.to_csv('IMDb_data.csv', index=False)

Nu de missende waarden zijn verwerkt, kunnen we door naar het goed opschonen van de data. Voor de films dataset is dit al gebeurt na het inladen van de data. Voor de andere datasets zal deze stap nu worden ondernomen. Er zal worden begonnen met het dataframe 'df'.

Om te bepalen welke stappen nodig zijn zal het dataframe eerst worden getoond.

In [11]:
# Tonen van de eerste vijf regels van df
display(df.head())

,Wikipedia_ID,Film,Genres
0,975900,Ghosts of Mars,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,"{""/m/07s9rl0"": ""Drama""}"


De kolommen die kunnen worden opgeschoond zijn de 'Film' kolom en de 'Genres' kolom. De 'Film' kolom zal later worden opgeschoond met een groter aantal tekst gebaseerde kolommen die zich in elke dataset bevind. De 'Genres' kolom zal worden opgeschoond door de values uit de dictionary te halen.

In [12]:
def process_text_columns(df, kolommen):
    """
    Deze functie processed de opgegeven kolommen, zodat de
    tekst bruikbaarder is. De opgegeven kolommen moeten
    hiervoor wel tekstuele data bevatten.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het dataframe waarop de aanpassingen worden uitgevoerd.

    kolommen : list or str
        De kolom(men) waarop de aanpassingen worden uitgevoerd.

    Returns:
    ----------
    df_nlp : pandas.DataFrame
        Het dataframe waarop de toepassing zijn uitgevoert.
    """
    # Het selecteren van engelse stopwoorden voor in de tekst
    stopwoorden = set(stopwords.words('english'))

    # Aanmaken van process text functie om apply te kunnen gebruiken
    def process_text(text):
        """
        Deze functies past verschillende taken
        toe op basis van NLP technieken

        Parameters:
        ----------
        text : str
            Een str aan tekst in een dataframe kolom

        Returns:
        ----------
        processed_text : str
            Een str met de aangepaste tekst
        """
        # Aanmaken van tokens in de tekst
        tokens = re.findall(r'\w+', text)

        # Het weghalen van punctuatie binnen de tekst
        no_punctuations = ' '.join(re.sub(r'\W', ' ', token) for token in tokens)

        # Het veranderen van alle tekst naar kleine letters
        lower_text = no_punctuations.lower()

        # Het verwijderen van de stopwoorden
        processed_text = ' '.join(word for word in lower_text.split() if word not in stopwoorden)

        return processed_text

    # Loopen over de kolommen van het opgegeven dataframe
    for kol in kolommen:
        df[kol] = df[kol].apply(process_text)

    return df

In [13]:
# Invoeren van de kolommen die aangepast worden
kolommen = ['Summary']

# Uitvoeren van de aanpassingen
processed_df = process_text_columns(df_sum, kolommen)

# Tonen van het aangepaste dataframe
display(processed_df)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Gebruiker/nltk_data'
    - 'C:\\Users\\Gebruiker\\anaconda3\\nltk_data'
    - 'C:\\Users\\Gebruiker\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Gebruiker\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Gebruiker\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Feature Engineering</div>
</div>

Nu alle data is gepreprocessed, kan er worden gewerkt aan het maken van de features. Een groot onderdeel bij Feature Engineering is het toepassen van de juiste technieken op de juiste typen data.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Opzetten van de Pipeline</div>
    <a name='pipe'></a>
</div>

Om te zorgen dat alle data op de juiste manier word ingeladen en getransformeerd maken we gebruik van een pipeline. De pipeline is eigenlijk een class vol met functies die ons helpt om data gemakkelijk in te laden en te transformeren. De functies in de pipeline zijn zo robuust mogelijk opgesteld, zodat er geen problemen zijn bij het inladen en verwerken van een andere databron.

In [ ]:
class ETL_Pipeline:
    """
    Class voor een Extract-Transform-Load Pipeline.
    Deze class kan nodige data van een database, API of
    webscraping bron halen. Deze data kan vervolgens
    getransformeerd worden, waarna de data kan worden
    ingeladen in een pandas DataFrame
    """
    def __init__(self, db_source=None, query=None, api_source=None, csv_file=None, headers=None, querystring=None):
        """
        Initiator van de class. De initiator neemt bepaalde
        waarden op. Met behulp van deze waarden kunnen de
        functies worden uitgevoerd.
        """
        self.db_source = db_source
        self.query = query
        self.api_source = api_source
        self.csv_file = csv_file
        self.data_frame = pd.DataFrame()
        self.headers = headers
        self.querystring = querystring

    def extract_db(self):
        """
        Deze functie haalt data op uit een lokale database.
        """
        # Verbinden met de database
        conn = sqlite3.connect(self.db_source)

        # Uitvoeren van de query op de database
        db_data = pd.read_sql_query(self.query, conn)

        # Sluiten van de verbinding met de database
        conn.close()

        return db_data

    def extract_api(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        response = requests.get(self.api_data, headers=self.headers, params=self.querystring)
        
        resultaten = []

        # Itereer over elk resultaat in de 'results' lijst van de JSON-respons
        for resultaat in response.json()['results']:
            # Haal de benodigde velden op
            titel = resultaat['titleText']['text'] if 'titleText' in resultaat else None
            genres = [genre['text'] for genre in resultaat['genres']['genres']] if 'genres' in resultaat else None
            plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else None

            # Voeg de resultaten toe aan de lijst
            resultaten.append({
                'Titel': titel,
                'Genre': genres,
                'Beschrijving': plot
            })
        
        api_data = pd.DataFrame(resultaten)

        return api_data

    def extract_csv(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        # Inlezen van csv data
        csv_data = pd.read_csv(self.csv_file)

        return csv_data

    def transform_data(self, data_frames):
        """
        Deze functie transformeerd de data op basis van
        de gegeven methoden.
        """
        transformed_data = pd.DataFrame(...)

        return transformed_data

    def load_data(self):
        """
        Deze functie laad de data in naar een
        pandas.DataFrame.
        """
        # Maken van lege data indien databron niet gebruikt
        db_data = None
        api_data = None
        csv_data = None

        # Extraheer data van gebruikte bronnen
        if self.db_source:
            db_data = self.extract_db()
        if self.api_source:
            api_data = self.extract_api()
        if self.csv_file:
            csv_data = self.extract_csv()

        # Zet alle databronnen in een lijst
        all_data_frames = [db_data, api_data, csv_data]

        # Transformeer en merge alle data
        transformed_data = self.transform_data(all_data_frames)

        # Toewijzen van de data aan de data_frame class attribuut
        self.data_frame = transformed_data

        return transformed_data

Nu de Pipeline is aangemaakt, kunnen de parameters worden gedefined. De parameters zijn opzettelijk los neergezet, om de Pipeline niet vast te zetten in het geval van kleine aanpassingen aan de locaties van bestanden.

In [15]:
# Invoeren van de DataBase naam
db_source = "movie_databse.db"

# Invoeren van query op DataBase
query = """
    SELECT mm.Film,
           ps.Summary,
           mm.Genres
    FROM 'movie.metadata' AS mm
        
    JOIN 'plot_summaries' AS ps
        ON mm.Wikipedia_ID = ps.Wikipedia_ID
"""

# Invoeren van de API link
api_source = "https://moviesminidatabase.p.rapidapi.com/movie/order/byRating/"

querystring = {"limit":"50","info":"base_info"}

headers = {
	"X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
	"X-RapidAPI-Host": "moviesminidatabase.p.rapidapi.com"
}

# Invoeren van csv file path
csv_file = "IMDb_data.csv"

Met alle nodige parameters gedefineerd, kan de pipeline worden gebruikt om de data in te laden.

In [17]:
etl = ETL_Pipeline(db_source, query, api_source, querystring, headers, csv_file)
df = etl.load_data()

TypeError: __init__() takes from 1 to 5 positional arguments but 7 were given

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Aantonen dat de Pipeline werkt</div>
    <a name='toon'></a>
</div>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Bronnen</div>
    <a name='bron'></a>
</div>

Please cite this paper if you write any papers involving the use of the data (van de Movie Summary Corpus):

    Learning Latent Personas of Film Characters
    David Bamman, Brendan O'Connor, and Noah A. Smith
    ACL 2013, Sofia, Bulgaria, August 2013

